In [ ]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Qt5Agg')
from mne.preprocessing import ICA
import mne
import numpy as np
import pandas as pd
import pyprep
import pyxdf
from utils import *
from scipy.signal import welch
import warnings
warnings.filterwarnings("ignore")

In [ ]:
xdf_filename = '/Users/bryan.gonzalez/CUNY_subs/sub-P5029423/sub-P5029423_ses-S001_task-CUNY_run-001_mobi.xdf'


In [ ]:
subject = xdf_filename.split('-')[1].split('/')[0]
df = get_event_data(event='RestingState', 
                    df=import_eeg_data(xdf_filename),
                    stim_df=import_stim_data(xdf_filename))

ch_names = [f"E{i+1}" for i in range(df.shape[1] - 1)]
info = mne.create_info(ch_names, 
                    sfreq=1/df.lsl_time_stamp.diff().mean(), 
                    ch_types='eeg')
df.drop(columns=['lsl_time_stamp'], inplace=True)

raw = mne.io.RawArray(df.T * 1e-6, info=info) # multiplying by 1e-6 converts to volts

# Create a Cz reference
value = np.zeros((1, raw.n_times))
info = mne.create_info(["Cz"], raw.info['sfreq'], ch_types='eeg')
cz = mne.io.RawArray(value, info)
raw.add_channels([cz], force_update_info=True)

# Apply a montage
montage = mne.channels.make_standard_montage('GSN-HydroCel-129')
raw.set_montage(montage, on_missing='ignore')

#raw.crop(tmin=0, tmax=5)

prep_params = {
        "ref_chs": "eeg",
        "reref_chs": "eeg",
        "line_freqs": np.arange(60, raw.info["sfreq"] / 2, 60),
    }
# these params set up the robust reference  - i.e. median of all channels and interpolate bad channels
prep = pyprep.PrepPipeline(raw, montage=montage, channel_wise=True, prep_params=prep_params)
prep_output = prep.fit()
raw_cleaned = prep_output.raw_eeg

In [ ]:
# set notch filter
raw_cleaned.notch_filter(60)
# set bandpass filter
raw_cleaned.filter(l_freq=1.0, h_freq=50.0) # only keeping frequencies between 1-50 Hz
# play around with this number to get components 
# that seem to represent the actual brain activations well
num_components = .95 
ica = ICA(n_components=num_components, method='picard')
ica.fit(raw_cleaned)

In [ ]:
eog_indices, scores = ica.find_bads_eog(raw_cleaned, threshold=3.0)
#ica.exclude.extend(eog_inds)


In [ ]:
# plot the components and wait for user input to select components
ica.plot_components( title='ICA Components')
#print("Select components to exclude (e.g., 0, 1, 2) and press Enter:")
#exclude = input().split(',')
#exclude = [int(i.strip()) for i in exclude if i.strip().isdigit()]

In [ ]:
ica.plot_sources(raw_cleaned)

In [ ]:
ica.plot_properties(raw_cleaned, picks=4) # This exact component number probably won't work if you recompute ICA


In [ ]:
ica.plot_overlay(raw_cleaned, exclude=[3,6,8])


In [ ]:
raw_cleaned.plot()

In [ ]:
raw_cleaned.save(f'./{subject}_{event_name}_cleaned_data.fif', overwrite=True)